In [ ]:
# Install this library


!pip install mtcnn==0.1.0
!pip install tensorflow==2.3.1
!pip install keras==2.4.3
!pip install keras-vggface==0.6
!pip install keras_applications==1.0.8

In [ ]:
!pip install -U flask_ngrok

In [ ]:
from google.colab.output import eval_js

print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://dhxkcfh8lc-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, request, render_template, flash, redirect, url_for
from flask_ngrok import run_with_ngrok
from werkzeug.utils import secure_filename
import os


import numpy as np
import cv2
from PIL import Image
import pickle
import os
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from mtcnn import MTCNN
from tensorflow.keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity

#==========================================================================

# Flask code start from here

UPLOAD_FOLDER = r"/content/drive/MyDrive/Python Flask/static/uploads/"

ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg'}


app = Flask(__name__, template_folder=r"/content/drive/MyDrive/Python Flask/templates/",static_folder=r"/content/drive/MyDrive/Python Flask/static")

run_with_ngrok(app=app)

app.secret_key = "secret_key"

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024


# Let's our pickle files

filenames_path = r"/content/drive/MyDrive/Python Flask/static/Files/filename_paths.pkl"

features_path = r"/content/drive/MyDrive/Python Flask/static/Files/all_features.pkl"

filenames = pickle.load(file=open(file=filenames_path, mode="rb"))

features = np.array(pickle.load(file=open(file=features_path, mode="rb")))

# Create model
model = VGGFace(include_top=False, model="resnet50", input_shape=(224,224,3), pooling='avg')

# Create our face detector object
detector = MTCNN()




def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route(rule="/")
def home():
    return render_template(template_name_or_list="home.html")


@app.route(rule="/", methods=["GET", "POST"])
def upload_file():

    similarity = []

    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            return redirect(request.url)

        file = request.files['file']

        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            # Join the image path where image will be save
            img_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            # Saving the image in uploads folder
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    
    # read the image using cv2
    input_img = cv2.imread(img_path)
    # Extract the face information
    face_points = detector.detect_faces(img=input_img)
    # Extract the only face points
    x, y, width, height = face_points[0]['box']
    # crop the pace part using face points
    face = input_img[y:y+height, x:x+width]
    # resize the image so that our model can take as a input
    resize_face = cv2.resize(src=face, dsize=(224,224))
    # Change the data type so that memory use less
    resize_face = resize_face.astype(dtype='float32')
    # we know we have to give a bacth input to deep learning model. so expand the dimension
    expanded_face = np.expand_dims(a=resize_face, axis=0)
    preprocessed_img = preprocess_input(x=expanded_face)
    predict_img = model.predict(x=preprocessed_img).flatten()

    # using for loop we are finding out all the cosign distance for every features vector.
    for i in range(len(features)):
        similarity.append(cosine_similarity(X=predict_img.reshape(1, -1), Y=features[i].reshape(1, -1))[0][0])

    index_pos = sorted(list(enumerate(similarity)), reverse=True, key=lambda x:x[1])[0][0]
    

    img_path = img_path.split("static/")[1]

    extract_path = filenames[index_pos][64:]

    actor_name = filenames[index_pos].split("/content/")[1].split("/")[2]

    all_folder_name = os.listdir(path=r"/content/drive/MyDrive/Python Flask/static/uploads2")

    if actor_name not in all_folder_name:
        img_path2 = r"uploads2/Not_found.png"
    else:
        img_name = os.listdir(path="/content/drive/MyDrive/Python Flask/static/uploads2/{}".format(actor_name))[0]
        img_path2 = f"uploads2/{actor_name}/{img_name}"


    print("Image path1: ",img_path)
    print("Image path2: ",img_path2)
    print("Index pos: ",index_pos)
    print("actor_name: ",actor_name)
    print("extract_path: ",extract_path)

        
    img1, img2 = img_path, img_path2

    actor_name = f"Celebrity Name: {actor_name}"

    return render_template(template_name_or_list='home.html', image1=img1, image2=img2, actorName=actor_name)
    # return render_template(template_name_or_list='home.html')



# @app.route(rule="/display/<filename>")
# def display_image(filename):
#     return redirect(location=url_for(r'/content/drive/MyDrive/Python Flask/static', filename='uploads/' + filename), code=301)


if __name__ == "__main__":
    app.run()